<a href="https://colab.research.google.com/github/basugautam/Reproducibility-Challenge-Project/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Load Dataset

dataset_path = r"C:\\LCTSF"
file_name = "timeseries_data.csv"
file_path = os.path.join(dataset_path, file_name)

if not os.path.isfile(file_path):
    raise FileNotFoundError(f"Dataset file not found at {file_path}. Ensure the file exists and the path is correct.")

data = pd.read_csv(file_path, parse_dates=["timestamp"], index_col="timestamp")

# Data Preprocessing
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 24  # Forecasting 24 time steps ahead
X, y = create_sequences(data_scaled, seq_length)
X_train, X_test = X[:int(0.8*len(X))], X[int(0.8*len(X)):]
y_train, y_test = y[:int(0.8*len(y))], y[int(0.8*len(y)):]

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

# Define Transformer Model
class TransformerForecaster(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super(TransformerForecaster, self).__init__()
        self.encoder = nn.Linear(input_dim, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.decoder = nn.Linear(d_model, input_dim)

    def forward(self, src):
        src = self.encoder(src)
        src = src.permute(1, 0, 2)  # Transformer expects seq_len, batch, feature
        output = self.transformer(src, src)
        output = output.permute(1, 0, 2)
        return self.decoder(output[:, -1, :])

# Initialize Model
model = TransformerForecaster(input_dim=X_train.shape[2])
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss function with Loss Shaping Constraints
def loss_shaping(y_pred, y_true):
    error = y_pred - y_true
    squared_error = torch.mean(torch.square(error))
    bound_loss = torch.mean(torch.clamp(error, min=-0.1, max=0.1))  # Constrain error bounds
    return squared_error + 0.5 * bound_loss

# Training Loop with Dynamic Constraint Adaptation
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_shaping(y_pred, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Evaluate Model
model.eval()
y_pred_test = model(X_test).detach().numpy()
y_test = y_test.numpy()
y_pred_test_rescaled = scaler.inverse_transform(y_pred_test)
y_test_rescaled = scaler.inverse_transform(y_test)

# Plot Results
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:100], label='Actual')
plt.plot(y_pred_test_rescaled[:100], label='Predicted')
plt.legend()
plt.show()


FileNotFoundError: Dataset file not found at C:\\LCTSF/timeseries_data.csv. Ensure the file exists and the path is correct.

In [4]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Load Dataset

dataset_path = r"C:\\LCTSF"
file_name = "timeseries_data.csv"
file_path = os.path.join(dataset_path, file_name)

if not os.path.isfile(file_path):
    raise FileNotFoundError(f"Dataset file not found at {file_path}. Ensure the file exists and the path is correct.")

data = pd.read_csv(file_path, parse_dates=["timestamp"], index_col="timestamp")

# Data Preprocessing
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 24  # Forecasting 24 time steps ahead
X, y = create_sequences(data_scaled, seq_length)
X_train, X_test = X[:int(0.8*len(X))], X[int(0.8*len(X)):]
y_train, y_test = y[:int(0.8*len(y))], y[int(0.8*len(y)):]

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

# Define Transformer Model
class TransformerForecaster(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super(TransformerForecaster, self).__init__()
        self.encoder = nn.Linear(input_dim, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.decoder = nn.Linear(d_model, input_dim)

    def forward(self, src):
        src = self.encoder(src)
        src = src.permute(1, 0, 2)  # Transformer expects seq_len, batch, feature
        output = self.transformer(src, src)
        output = output.permute(1, 0, 2)
        return self.decoder(output[:, -1, :])

# Initialize Model
model = TransformerForecaster(input_dim=X_train.shape[2])
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss function with Loss Shaping Constraints and Duality-based Optimization
def loss_shaping(y_pred, y_true):
    error = y_pred - y_true
    squared_error = torch.mean(torch.square(error))
    bound_loss = torch.mean(torch.clamp(error, min=-0.1, max=0.1))  # Constrain error bounds
    duality_term = torch.mean(torch.exp(-torch.abs(error)))  # Duality-based term
    return squared_error + 0.5 * bound_loss + 0.2 * duality_term

# Training Loop with Dynamic Constraint Adaptation
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_shaping(y_pred, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Evaluate Model
model.eval()
y_pred_test = model(X_test).detach().numpy()
y_test = y_test.numpy()
y_pred_test_rescaled = scaler.inverse_transform(y_pred_test)
y_test_rescaled = scaler.inverse_transform(y_test)

# Compute Evaluation Metrics
mse = np.mean((y_pred_test_rescaled - y_test_rescaled) ** 2)
std_error = np.std(y_pred_test_rescaled - y_test_rescaled)
print(f"Mean Squared Error (MSE): {mse}")
print(f"Standard Deviation of Errors: {std_error}")

# Plot Results
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:100], label='Actual')
plt.plot(y_pred_test_rescaled[:100], label='Predicted')
plt.legend()
plt.show()


FileNotFoundError: Dataset file not found at C:\\LCTSF/timeseries_data.csv. Ensure the file exists and the path is correct.

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import random

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Load Dataset

dataset_path = r"C:\\LCTSF"
file_name = "timeseries_data.csv"
file_path = os.path.join(dataset_path, file_name)

if not os.path.isfile(file_path):
    raise FileNotFoundError(f"Dataset file not found at {file_path}. Ensure the file exists and the path is correct.")

data = pd.read_csv(file_path, parse_dates=["timestamp"], index_col="timestamp")

# Data Preprocessing
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 24  # Forecasting 24 time steps ahead
X, y = create_sequences(data_scaled, seq_length)
X_train, X_test = X[:int(0.8*len(X))], X[int(0.8*len(X)):]
y_train, y_test = y[:int(0.8*len(y))], y[int(0.8*len(y)):]

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

# Define Transformer Model
class TransformerForecaster(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super(TransformerForecaster, self).__init__()
        self.encoder = nn.Linear(input_dim, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.decoder = nn.Linear(d_model, input_dim)

    def forward(self, src):
        src = self.encoder(src)
        src = src.permute(1, 0, 2)  # Transformer expects seq_len, batch, feature
        output = self.transformer(src, src)
        output = output.permute(1, 0, 2)
        return self.decoder(output[:, -1, :])

# Initialize Model
model = TransformerForecaster(input_dim=X_train.shape[2])
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss function with Loss Shaping Constraints and Duality-based Optimization
def loss_shaping(y_pred, y_true):
    error = y_pred - y_true
    squared_error = torch.mean(torch.square(error))
    bound_loss = torch.mean(torch.clamp(error, min=-0.1, max=0.1))  # Constrain error bounds
    duality_term = torch.mean(torch.exp(-torch.abs(error)))  # Duality-based term
    return squared_error + 0.5 * bound_loss + 0.2 * duality_term

# Training Loop with Dynamic Constraint Adaptation
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_shaping(y_pred, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Evaluate Model
model.eval()
y_pred_test = model(X_test).detach().numpy()
y_test = y_test.numpy()
y_pred_test_rescaled = scaler.inverse_transform(y_pred_test)
y_test_rescaled = scaler.inverse_transform(y_test)

# Compute Evaluation Metrics
mse = np.mean((y_pred_test_rescaled - y_test_rescaled) ** 2)
std_error = np.std(y_pred_test_rescaled - y_test_rescaled)
print(f"Mean Squared Error (MSE): {mse}")
print(f"Standard Deviation of Errors: {std_error}")

# Plot Results
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:100], label='Actual')
plt.plot(y_pred_test_rescaled[:100], label='Predicted')
plt.legend()
plt.show()


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import random

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Load Dataset
dataset_path = r"C:\\LCTSF"
file_name = "timeseries_data.csv"
file_path = os.path.join(dataset_path, file_name)

if not os.path.isfile(file_path):
    raise FileNotFoundError(f"Dataset file not found at {file_path}. Ensure the file exists and the path is correct.")

data = pd.read_csv(file_path, parse_dates=["timestamp"], index_col="timestamp")

# Data Preprocessing
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 24  # Forecasting 24 time steps ahead
X, y = create_sequences(data_scaled, seq_length)
X_train, X_test = X[:int(0.8*len(X))], X[int(0.8*len(X)):]
y_train, y_test = y[:int(0.8*len(y))], y[int(0.8*len(y)):]

# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

# Define Transformer Model with Loss Shaping Constraints and Optimization
class TransformerForecaster(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super(TransformerForecaster, self).__init__()
        self.encoder = nn.Linear(input_dim, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.decoder = nn.Linear(d_model, input_dim)

    def forward(self, src):
        src = self.encoder(src)
        src = src.permute(1, 0, 2)  # Transformer expects seq_len, batch, feature
        output = self.transformer(src, src)
        output = output.permute(1, 0, 2)
        return self.decoder(output[:, -1, :])

# Initialize Model
model = TransformerForecaster(input_dim=X_train.shape[2])
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss Function with Loss Shaping Constraints and Primal-Dual Optimization
def loss_shaping(y_pred, y_true):
    error = y_pred - y_true
    squared_error = torch.mean(torch.square(error))
    bound_loss = torch.mean(torch.clamp(error, min=-0.1, max=0.1))  # Constrain error bounds
    duality_term = torch.mean(torch.exp(-torch.abs(error)))  # Primal-Dual optimization term
    return squared_error + 0.5 * bound_loss + 0.2 * duality_term

# Training Loop with Adaptive Constraints
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_shaping(y_pred, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Evaluate Model
model.eval()
y_pred_test = model(X_test).detach().numpy()
y_test = y_test.numpy()
y_pred_test_rescaled = scaler.inverse_transform(y_pred_test)
y_test_rescaled = scaler.inverse_transform(y_test)

# Compute Evaluation Metrics
mse = np.mean((y_pred_test_rescaled - y_test_rescaled) ** 2)
std_error = np.std(y_pred_test_rescaled - y_test_rescaled)
print(f"Mean Squared Error (MSE): {mse}")
print(f"Standard Deviation of Errors: {std_error}")

# Plot Results
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled[:100], label='Actual')
plt.plot(y_pred_test_rescaled[:100], label='Predicted')
plt.legend()
plt.show()
